<a href="https://colab.research.google.com/github/train-hong/metro-json/blob/main/%E5%A4%A7%E6%95%B8%E6%93%9A%E6%87%89%E7%94%A8%E5%B0%88%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
from datetime import datetime
import pytz, sys, requests

stas = requests.get("https://raw.githubusercontent.com/train-hong/metro-json/main/TRTC.json").json() #站名來源

sta = {} #key = 路線, value = {流水號: 車站編號, 站名}
for i in stas:
  sta[i["LineNo"]] = {}
  for j in i["Stations"]:
    sta[i["LineNo"]][j["Sequence"]] = [j["StationID"], j["StationName"]["Zh_tw"]]
Luzhou = [13, 14, 16, 19, 20]
Huilong = [12, 15, 17, 18, 21, 22, 23, 24, 25]
for i in range(13, 22):
  del sta["O"][i]
for i in range(13, 18):
  Lz = stas[3]["Stations"][Luzhou[i-13]]
  sta["O"][i] = [Lz["StationID"], Lz["StationName"]["Zh_tw"]]
for i in range(18, 27):
  Hl = stas[3]["Stations"][Huilong[i-18]]
  sta["O"][i] = [Hl["StationID"], Hl["StationName"]["Zh_tw"]]
del sta["G"][20]
del sta["R"][28]

print("1 淡水信義線\n2 松山新店線\n3 板南線\n4 中和新蘆線")
line = int(input("Please enter your line number: "))
print('')
st = [] #車站時刻表(station timetable)
ln = ["R", "G", "BL", "O"] #linenumber
tt = requests.get(f"https://raw.githubusercontent.com/ericyu/TaipeiMetroTimeTable/master/Lines/{ln[line-1]}.json").json() #時刻表(timetable)
w = int(datetime.now(pytz.timezone('ROC')).strftime("%w")) #現在星期幾
wq = [2, 0, 0, 0, 0, 0, 1] #星期幾轉換為要引用的時刻表(weekday quote)
bwq = [3, 0, 0, 0, 0, 1, 2] #板南線引用的時刻表(blue weekday quote)

if line == 1:
  print("1 淡水\n2 象山")
  di = int(input("Please enter your direction number: ")) #red line direction
  print('')
  for i in sta[ln[0]]:
    print(i, sta[ln[0]][i][1])
  sn = int(input("Please enter your station number: ")) #station now
  print('')  
  if sn == 1 and di == 2:
    sys.exit("你已經在淡水囉")
  elif sn == 27 and di == 1:
    sys.exit("你已經在象山囉")
  for i in tt[di-1]["Timetables"][wq[w]]["Trains"]:
    for j in i["Schedule"]:
      if j["StationCode"] == sta["R"][sn][0]:
        st.append(j)

elif line == 2:
  print("1 新店\n2 松山")
  di = int(input("Please enter your direction number: ")) #green line direction
  print('')
  for i in sta[ln[1]]:
    print(i, sta[ln[1]][i][1])
  sn = int(input("Please enter your station number: "))
  print('')
  if sn == 1 and di == 1:
    sys.exit("你已經在松山囉")
  elif sn == 19 and di == 2:
    sys.exit("你已經在新店囉")
  for i in tt[di-1]["Timetables"][wq[w]]["Trains"]:
    for j in i["Schedule"]:
      if j["StationCode"] == sta["G"][sn][0]:
        st.append(j)

elif line == 3:
  print("1 南港展覽館\n2 頂埔")
  di = int(input("Please enter your direction number: "))
  print('')
  for i in sta[ln[2]]:
    print(i, sta[ln[2]][i][1])
  sn = int(input("Please enter your station number: "))
  print('')
  if sn == 23 and di == 1:
    sys.exit("你已經在南港展覽館囉")
  elif sn == 1 and di == 2:
    sys.exit("你已經在頂埔囉")
  for i in tt[di-1]["Timetables"][bwq[w]]["Trains"]:
    for j in i["Schedule"]:
      if j["StationCode"] == sta["BL"][sn][0]:
        st.append(j)

elif line == 4:
  print("1 南勢角\n2 蘆洲、迴龍")
  di = int(input("Please enter your direction number: "))
  print('')
  for i in sta[ln[3]]:
    print(i, sta[ln[3]][i][1])
  sn = int(input("Please enter your station number: "))
  print('')
  if sn == 1 and di == 1:
    sys.exit("你已經在南勢角囉")
  elif sn == 17 and di == 2:
    sys.exit("你已經在蘆洲囉")
  elif sn == 26 and di == 2:
    sys.exit("你已經在迴龍囉")
  for i in tt[di-1]["Timetables"][wq[w]]["Trains"]:
    for j in i["Schedule"]:
      if j["StationCode"] == sta["O"][sn][0]:
        st.append(j)

#h = int(datetime.now(pytz.timezone('ROC')).strftime("%-H")) #現在幾點（小時）
#m = int(datetime.now(pytz.timezone('ROC')).strftime("%-M")) #現在幾點（分鐘）
tn = 8*60 + 35 #time now 
jt = []
dt = []
for i in st:
  dt.append(i["DepTime"])
for i in dt:
  jt.append((int(i[0])*10 + int(i[1])) * 60 + int(i[3])*10 + int(i[4]))
jt.sort()
if tn > jt[-1]:
  tn = tn-1440
jt.append(tn)
jt.sort()
tnp = jt.index(tn) #time now position
if jt[tnp+1] == tn:
  at = 0 #arrive time
else:
  at = jt[tnp+1] - tn

print('{}站往{}方向列車 距離下次發車還有{}分鐘'.format(sta[ln[line-1]][sn][1], tt[di-1]["Direction"], at))

1 淡水信義線
2 松山新店線
3 板南線
4 中和新蘆線
Please enter your line number: 3

1 南港展覽館
2 頂埔
Please enter your direction number: 1

1 頂埔
2 永寧
3 土城
4 海山
5 亞東醫院
6 府中
7 板橋
8 新埔
9 江子翠
10 龍山寺
11 西門
12 台北車站
13 善導寺
14 忠孝新生
15 忠孝復興
16 忠孝敦化
17 國父紀念館
18 市政府
19 永春
20 後山埤
21 昆陽
22 南港
23 南港展覽館
Please enter your station number: 18

市政府站往南港展覽館方向列車 距離下次發車還有2分鐘


In [ ]:
11
111111111111111111111111121

11111111111111121

週一～五<br>
往北投 21站 象山～北投 0-50 51台車<br>
往淡水 27站 象山～淡水 51-200 150台車<br>
往北投 18站 大安～北投 201-287 87台車<br>
往淡水 24站 大安～淡水 288-291 4台車<br>
往淡水 19站 台北車站～淡水 292 1台車<br>
往淡水 14站 劍潭～淡水 293 1台車<br>
往淡水 9站 唭哩岸～淡水 294 1台車<br>
往淡水 5站 忠義～淡水 295-305 11台車<br>
往淡水 2站 紅樹林～淡水 306 1台車<br><br>
週六<br>
往北投 21站 象山～北投 0-2<br>
往淡水 27站 象山～淡水 3-123<br>
往北投 18站 大安～北投 124-225<br>
往淡水 24站 大安～淡水 226-229<br>
往淡水 19站 台北車站～淡水 230<br>
往淡水 14站 劍潭～淡水 231<br>
往淡水 9站 唭哩岸～淡水 232<br>
往淡水 5站 忠義～淡水 233-236<br>
往淡水 2站 紅樹林～淡水 237<br><br>
週日<br>
往北投 21站 象山～北投 0-2<br>
往淡水 27站 象山～淡水 3-123<br>
往北投 18站 大安～北投 124-225<br>
往淡水 24站 大安～淡水 226-229<br>
往淡水 19站 台北車站～淡水 230<br>
往淡水 14站 劍潭～淡水 231<br>
往淡水 9站 唭哩岸～淡水 232<br>
往淡水 5站 忠義～淡水 233-236<br>
往淡水 2站 紅樹林～淡水 237<br><br>